a) lmbda : is used for regularization, prevents overfitting
   deg : degree of polynomial, determines until which degree the phi would be
   flagSTD : flags used to determined if the data needs scaling or not
   without rescaling then the data could be far off from each other reducing the accuracy of the prediction result
   
b) K : how many nearest neighbors
   flagKLinReg : flags used to determine wether to do linear least squares or take the average of the KNN
   
   
data accuracy is to be determined by comparing my result from the program with the result of airfoil

In [1]:
#!/usr/bin/env python
# V2A3_regression_airfoilnoise.py
# Programmgeruest zu Versuch 2, Aufgabe 3
# to log outputs start with: python V2A3_regression_airfoilnoise.py >V2A3_regression_airfoilnoise.log

import numpy as np
import pandas as pd

#from V2A2_Regression import *
from V2A2 import *

# ***** MAIN PROGRAM ********
# (I) Hyper-Parameters
S=3;               # S-fold cross-validation
lmbda=1;           # regularization parameter (lambda>0 avoids also singularities)
K=1;               # K for K-Nearest Neighbors
flagKLinReg = 0;   # if flag==1 and K>=D then do a linear regression of the KNNs to make prediction
deg=1;             # degree of basis function polynomials 
flagSTD=0;         # if >0 then standardize data before training (i.e., scale X to mean value 0 and standard deviation 1)
N_pred=5;          # number of predictions on the training set for testing
x_test_1 = [0,0,0,0,0];   # REPLACE dummy code: define test vector 1
x_test_2 = [0,0,0,0,0];   # REPLACE dummy code: define test vector 2

# (II) Load data 
fname=('../DATA/AirfoilSelfNoise/airfoil_self_noise.xls')
airfoil_data = pd.read_excel(fname,0); # load data as pandas data frame 
T = airfoil_data.values[:,5]           # target values = noise load (= column 5 of data table)
X = airfoil_data.values[:,:5]          # feature vectors (= column 0-4 of data table)
N,D=X.shape                            # size and dimensionality of data set
idx_perm = np.random.permutation(N)    # get random permutation for selection of test vectors 
print ("Data set ",fname," has size N=", N, " and dimensionality D=",D)
print ("X=",X)
print ("T=",T)
print ("x_test_1=",x_test_1)
print ("x_test_2=",x_test_2)
print ("number of basis functions M=", len(phi_polynomial(X[1],deg)))

# (III) Do least-squares regression with regularization 
print ("\n#### Least Squares Regression with regularization lambda=", lmbda, " ####")
phi=lambda X: phi_polynomial(X,deg)
lsr = LSRRegressifier(lmbda,phi)
lsr.fit(X,T)  # REPLACE dummy code: Create and fit Least-Squares Regressifier using polynomial basis function of degree deg and flagSTD for standardization of data  
print ("lsr.W_LSR=", lsr.W_LSR)   # REPLACE dummy code: print weight vector for least squares regression  
print ("III.1) Some predictions on the training data:")
for i in range(N_pred): 
    n=idx_perm[i]
    print ("Prediction for X[",n,"]=",X[n]," is y=",lsr.predict(X[n],flagSTD),", whereas true value is T[",n,"]=",T[n])   # REPLACE dummy code: compute prediction for X[n]
print ("III.2) Some predicitions for new test vectors:")
print ("Prediction for x_test_1 is y=", lsr.predict(x_test_1,flagSTD))    # REPLACE dummy code: compute prediction for x_test_1
print ("Prediction for x_test_2 is y=", lsr.predict(x_test_2,flagSTD))    # REPLACE dummy code: compute prediction for x_test_2
print ("III.3) S=",S,"fold Cross Validation:")
err_abs,err_rel = lsr.crossvalidate(S,X,T)# REPLACE dummy code: do cross validation!! 
print ("absolute errors (E,sd,min,max)=", err_abs, "\nrelative errors (E,sd,min,max)=", err_rel) 

# (IV) Do KNN regression  
print ("\n#### KNN regression with flagKLinReg=", flagKLinReg, " ####")
knnr = KNNRegressifier(K)     # REPLACE dummy code: Create and fit KNNRegressifier
knnr.fit(X,K)
print ("IV.1) Some predictions on the training data:")
for i in range(N_pred): 
    n=idx_perm[i]
    print ("Prediction for X[",n,"]=",X[n]," is y=",knnr.predict(X[n],K,flagKLinReg),", whereas true value is T[",n,"]=",T[n])   # REPLACE dummy code: compute prediction for X[n]
print ("IV.2) Some predicitions for new test vectors:")
print ("Prediction for x_test_1 is y=", knnr.predict(x_test_1,K,flagKLinReg))    # REPLACE dummy code: compute prediction for x_test_1
print ("Prediction for x_test_2 is y=", knnr.predict(x_test_2,K,flagKLinReg))    # REPLACE dummy code: compute prediction for x_test_2
print ("IV.3) S=",S,"fold Cross Validation:")
err_abs,err_rel = knnr.crossvalidate(S,X,T)                   # REPLACE dummy code: do cross validation!! 
print ("absolute errors (E,sd,min,max)=", err_abs, "\nrelative errors (E,sd,min,max)=", err_rel) 

Data set  ../DATA/AirfoilSelfNoise/airfoil_self_noise.xls  has size N= 1502  and dimensionality D= 5
X= [[1.00000e+03 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]
 [1.25000e+03 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]
 [1.60000e+03 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]
 ...
 [4.00000e+03 1.56000e+01 1.01600e-01 3.96000e+01 5.28487e-02]
 [5.00000e+03 1.56000e+01 1.01600e-01 3.96000e+01 5.28487e-02]
 [6.30000e+03 1.56000e+01 1.01600e-01 3.96000e+01 5.28487e-02]]
T= [125.201 125.951 127.591 ... 106.604 106.224 104.204]
x_test_1= [0, 0, 0, 0, 0]
x_test_2= [0, 0, 0, 0, 0]
number of basis functions M= 6

#### Least Squares Regression with regularization lambda= 1  ####
lsr.W_LSR= [ 1.31004588e+02 -1.23902426e-03 -6.06620941e-01 -3.35499467e+01
  1.21621508e-01 -1.51100354e+01]
III.1) Some predictions on the training data:
Prediction for X[ 1187 ]= [1.25000e+03 1.74000e+01 2.54000e-02 3.96000e+01 1.72206e-02]  is y= None , whereas true value is T[ 1187 ]= 131.364
Predic

IndexError: too many indices for array